In [ ]:
%%html
<style> table { display: inline-block } </style>

# CAB420 Assigment 2 - Model 4 - Litian

| Name   | Student Number |
| :---   | :--- |
| Charmi Raval       | N10510702          |
| Leonardo Villamil  | N10411526          |
| Litian Chen        | N9646922           |
| Tace Stewart       | N9654411           |

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import visualkeras

from sklearn.utils import class_weight
from sklearn import preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, concatenate, BatchNormalization, SpatialDropout2D, Activation, Flatten, AveragePooling2D, Dropout
from tensorflow.keras.models import Model
from sklearn.utils import class_weight
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.applications import ResNet50

## Load Data

In [ ]:
# Load data
train = pd.read_csv ('Data/training.csv')
validation = pd.read_csv ('Data/validation.csv')
test = pd.read_csv ('Data/testing.csv')

In [ ]:
# Split filename, features and label

X_train = train.drop(['filename', 'accent'], axis=1)
path_train = train['filename'].to_numpy()
y_train = train['accent'].to_numpy()

X_val = validation.drop(['filename', 'accent'], axis=1)
path_val = validation['filename'].to_numpy()
y_val = validation['accent'].to_numpy()

X_test = test.drop(['filename', 'accent'], axis=1)
path_test = test['filename'].to_numpy()
y_test = test['accent'].to_numpy()

In [ ]:
# Encode strings into class numbers
labels = np.unique(y_train)

encoder = preprocessing.LabelEncoder()
encoder.fit(y_train)

y_train = encoder.transform(y_train)
y_val = encoder.transform(y_val)
y_test = encoder.transform(y_test)

In [ ]:
# Calculation of class weights
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

for i in range(17):
    print('%s - class %s - weight %s\n' %(labels[i], (str(np.unique(y_train)[i])), str(class_weights[i])))

In [ ]:
datagen = ImageDataGenerator()

# load and iterate training dataset
train_it = datagen.flow_from_directory('Data/spectograms/train/',
                                       target_size=(224, 224),
                                       classes=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16'],
                                       class_mode='categorical', batch_size=64, seed=1)

# load and iterate validation dataset
val_it = datagen.flow_from_directory('Data/spectograms/validation/',
                                     target_size=(224, 224),
                                     classes=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16'], 
                                     class_mode='categorical', batch_size=64, seed=1)

# load and iterate test dataset
test_it = datagen.flow_from_directory('Data/spectograms/test/',
                                      target_size=(224, 224),
                                      classes=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16'],
                                      class_mode='categorical', batch_size=1,
                                      shuffle=False)

## Fine-tuning 

In [ ]:
def eval_model(prediction, test_it, labels, history):
    test_it.reset()
    print('Classification Report\n')
    print(classification_report(test_it.classes, prediction))

    test_it.reset()
    print('Confusion Matrix')
    cm = confusion_matrix(test_it.classes, prediction, normalize='true')
    fig = plt.figure(figsize=[20, 20])
    ax = fig.add_subplot(1, 1, 1)
    c = ConfusionMatrixDisplay(cm, display_labels=labels)
    c.plot(ax = ax);

    fig = plt.figure(figsize=[12, 4])
    ax = fig.add_subplot(1, 2, 1)
    ax.plot(history.history['loss'], label="Training Loss")
    ax.plot(history.history['val_loss'], label="Validation Loss")
    ax.legend()

    ax = fig.add_subplot(1, 2, 2)
    ax.plot(history.history['accuracy'], label="Training Accuracy")
    ax.plot(history.history['val_accuracy'], label="Validation Accuracy")
    ax.legend();

    fig = plt.figure(figsize=[12, 4])
    ax = fig.add_subplot(1, 2, 1)
    ax.plot(history.history['precision'], label="Training Precision")
    ax.plot(history.history['val_precision'], label="Validation Precision")
    ax.legend()

    ax = fig.add_subplot(1, 2, 2)
    ax.plot(history.history['recall'], label="Training Recall")
    ax.plot(history.history['val_recall'], label="Validation Recall")
    ax.legend();

In [ ]:
ResNet = ResNet50()
print(ResNet.summary())
plot_model(ResNet, show_shapes=True)

In [ ]:
visualkeras.layered_view(ResNet, legend=True)

In [ ]:
baseModel = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
print(baseModel.summary())

In [ ]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(17, activation="softmax")(headModel)

cnn_model3_litian = Model(inputs=baseModel.input, outputs=headModel, name='cnn_model3')
print(cnn_model3_litian.summary())

In [ ]:
# Freeze the base model layers
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
cnn_model3_litian.compile(optimizer=keras.optimizers.Adam(),
                   loss='categorical_crossentropy',
                   loss_weights=class_weights,
                   metrics=['accuracy', keras.metrics.Precision(name='precision'), keras.metrics.Recall(name='recall')])

In [ ]:
load = False

if load:
    cnn_model3_litian = load_model('Models/cnn_model3_litian')
else:
    history = cnn_model3_litian.fit(train_it,
                            steps_per_epoch=1395,
                            epochs=25,
                            validation_data=val_it,
                            validation_steps=465)

    # Save model
    cnn_model3_litian.save('Models/cnn_model3_litian')

In [ ]:
test_it.reset()
test_scores = cnn_model3_litian.evaluate(test_it, steps=29759)

test_it.reset()
prediction = cnn_model3_litian.predict(test_it, steps=29759)
prediction = np.argmax(prediction, axis=1)

eval_model(prediction, test_it, labels, history)